CBOW

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Thesis/posts1352_standard.csv")

In [ ]:
df

,No,Category,SubCategory,Post,HateSpeech,PreprocessedToken
0,1,Race,Chinese,Chinese are pigs!,1,"['chinese', 'pigs']"
1,2,Race,Indian,I don't like pajeets they smell like curry,1,"['unalike', 'pajeets', 'smell', 'like', 'curry']"
2,4,Race,Indian,"Indians always smell stinky and weird, what is...",1,"['indians', 'smell', 'stinky', 'weird']"
3,5,Gender,Female,Women are so emotional they can't handle anyth...,1,"['women', 'emotional', 'handle', 'anything', '..."
4,6,Race,Malay,Malays are dependant on subsidiaries from the ...,1,"['malays', 'dependant', 'subsidiaries', 'gover..."
...,...,...,...,...,...,...
1615,1349,Religion,Buddhism,You cannot build a temple in an area with majo...,1,"['build', 'temple', 'area', 'majority', 'musli..."
1616,1350,Religion,Buddhism,"Muslims should not enter temple, or they might...",1,"['muslims', 'exit', 'temple', 'might', 'accide..."
1617,1351,Gender,Male,Men that don't have job are useless piece of s...,1,"['men', 'lack', 'job', 'useless', 'piece', 'sh..."
1618,1352,Gender,Female,Women are dick machines.,1,"['women', 'dick', 'machines']"


In [ ]:
#obtain token list that's actually list instead of string
from ast import literal_eval

tokenlist = []
for x in df['PreprocessedToken']:
    tokenlist.append(' '.join(literal_eval(x)))

In [ ]:
from gensim.models import Word2Vec
from sklearn.naive_bayes import MultinomialNB, BernoulliNB , GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

corpus = [token.split() for token in tokenlist]

In [ ]:
# Train CBOW model
cbow_model = Word2Vec(corpus, vector_size=100, window=3, min_count=1, sg=0) #the best one
skipgram_model = Word2Vec(corpus, vector_size=100, window=3, min_count=1, sg=1) # doesn't have great accuracy

vocab = set(cbow_model.wv.index_to_key)

In [ ]:
cbow_model.save("/content/drive/MyDrive/Thesis/posts1352_standard_cbow_model")
skipgram_model.save("/content/drive/MyDrive/Thesis/posts1352_standard_skipgram_model")

#loaded_model = Word2Vec.load("word2vec_cbow.model")

In [ ]:
def cbow_vector(doc):
    # Remove out-of-vocabulary words
    doc = [word for word in doc if word in vocab]
    return np.mean(cbow_model.wv[doc], axis=0)

def skipgram_vector(doc):
    # Remove out-of-vocabulary words
    doc = [word for word in doc if word in vocab]
    return np.mean(skipgram_model.wv[doc], axis=0)

# Transform corpus into document vectors
doc_vectors = np.array([cbow_vector(doc) for doc in corpus])

skipgram_doc_vectors = np.array([skipgram_vector(doc) for doc in corpus])

In [ ]:
doc_vectors.shape

(1620, 100)

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(doc_vectors, df['HateSpeech'], test_size=0.3)

# Initialize and train the Naive Bayes classifier
Gaussian_classifier = GaussianNB()
Gaussian_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = Gaussian_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.602880658436214


In [ ]:
# Initialize and train the Naive Bayes classifier
nb_classifier = BernoulliNB()
nb_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = nb_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

#Cannot do MultinomialNB as it has negative value

Accuracy: 0.565843621399177


In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(skipgram_doc_vectors, df['HateSpeech'], test_size=0.3)

# Initialize and train the Naive Bayes classifier
Gaussian_classifier = GaussianNB()
Gaussian_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = Gaussian_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.5987654320987654


In [ ]:
# Initialize and train the Naive Bayes classifier
nb_classifier = BernoulliNB()
nb_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = nb_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

#Cannot do MultinomialNB as it has negative value

Accuracy: 0.5473251028806584


In [ ]:
import pickle

# Save the naive bayes model to a file using pickle
with open('/content/drive/MyDrive/Thesis/Gaussian_CBOW_model.pkl', 'wb') as file:
    pickle.dump(Gaussian_classifier, file)

# To load the model from the file
with open('/content/drive/MyDrive/Thesis/Gaussian_CBOW_model.pkl', 'rb') as file:
    loaded_model_pickle = pickle.load(file)